In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from qat.utils.pydref import HasId, Ref, RefDict, RefList

In [3]:
from qat.utils.pydantic import WarnOnExtraFieldsModel

from pydantic import (
    PlainSerializer,
)

from typing import Dict, List, Optional
import typing
from typing import Annotated

componentid = str


In [12]:
class At(HasId):
    x: int
    
class Bt(HasId):
    x: int
    s: str
    As: RefDict[At]

class Ct(HasId):
    x: int
    As: RefDict[At]
    Bs: RefList[Bt]
    someB: Ref[Bt]

class Dt(HasId):
    x: int
    Cs: RefDict[Ct]

In [13]:
class Outer(HasId):
    A: list[At]
    B: list[Bt]
    C: list[Ct]
    D: list[Dt]

This model is a DAG which means it can be serialised in layers

In [14]:
pick = lambda L, size=3: {l.id: l for l in np.random.choice(L, size=size)}

A = [At(x=i, id='A' + str(i)) for i in range(10)]
B = [Bt(x=i, id='B' + str(i), As=pick(A,3), s='blah' + str(i)) for i in range(10)]
C = [Ct(x=i, id='C' + str(i), As=pick(A,3), Bs=list(pick(B,3).values()), someB=list(pick(B,1).values())[0]) for i in range(10)]
D = [Dt(x=i, id='D' + str(i), Cs=pick(C,3)) for i in range(5)]

In [15]:
O1 = Outer(A=A, B=B,C=C,D=D, id='outer')
blob = O1.model_dump()

In [16]:
O2 = Outer(**blob)
assert(O2 == O1)